In [126]:
import numpy as np

from numpy.testing import assert_allclose

from numpy.linalg import norm

In [127]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim) 
    n = vec.shape[0]
    y = np.zeros(n)
    y[0] = norm(vec)
    u = (vec - y) / norm(vec - y)
    H = np.eye(n) - 2 * np.outer(u, u)
    return y, H

In [128]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol=1e-10)
assert_allclose(np.dot(h, v), v1, atol=1e-10)

In [129]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

In [163]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    R_matrix = a1.copy()
    H_matrix = np.diag([1.]*m)
    for i in range(n):
        vec = R_matrix[i:, i]
        y, H = householder(vec)
        H_i = np.diag([1.]*m)
        H_i[i:, i:] = H
        H_matrix = np.dot(H_matrix, H_i)
        R_matrix = np.dot(H_i, R_matrix)
    return H_matrix, R_matrix

In [164]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [166]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

In [167]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

QR-разложение совпадает с точностью до знака, поскольку не обязано быть единственным.

In [169]:
matrix_list = [a, q, qq, r, rr]
for matrix in matrix_list:
    print(matrix, '\n')

[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]] 

[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 

[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.        

In [173]:
def qr_decomp_upd(a):
    rv = []
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    R_matrix = a1.copy()
    for i in range(n):
        vec = R_matrix[i:, i]
        n = vec.shape[0]
        y = np.zeros(n)
        y[0] = norm(vec)
        u = (vec - y) / norm(vec - y)
        R_matrix[i:, i:] = R_matrix[i:, i:] - 2 * np.outer(u, np.dot(u, R_matrix[i:, i:]))
        rv.append(u)
    return R_matrix, rv

In [188]:
def q(a, rv):
    m, n = a.shape
    q = np.diag([1.]*m)
    for j in range(n-1, -1, -1):
        q[j:, j:] = q[j:, j:] - 2 * np.outer(rv[j], np.dot(rv[j], q[j:, j:])) 
    return q

In [189]:
r, rv = qr_decomp_upd(a)
q = q(a, rv)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)